In [ ]:
!pip install --upgrade tensorflow-federated
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [5]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    sys.path.append("./drive/MyDrive/Colab Notebooks/Projektarbeit_2/")
    BASE_DIR = sys.path[-1]
    from google.colab import files
    files.upload()

else: 
    BASE_DIR = "../"
    sys.path.append(BASE_DIR)

import json
import numpy as np 
import pandas as pd
import tensorflow as tf
# import tensorflow_federated as tff
from sklearn.model_selection import train_test_split
import collections
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Softmax
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import random
import time
from Reader import Reader
from FLModel import FLModel
from Utils import Utils
import logging

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
def read_config():
    config_file = BASE_DIR + "config/config.json"
    config = None
    with open(config_file) as json_file:
        config = json.loads(json_file.read())
    return config

def split_input_target(input, target):
    return input, target

def create_dataset(x, y, use_tff = True):
    ds =  tf.data.Dataset.from_tensor_slices((x, y))
    
    if use_tff:
        return (
        # Shuffle and form minibatches
        ds.repeat(EPOCHS).shuffle(SHUFFLE_BUFFER)
        # And finally split into (input, target) tuples
        .map(split_input_target)).batch(BATCH_SIZE) 
    else:
        return ds.shuffle(SHUFFLE_BUFFER).batch(BATCH_SIZE) 

def get_split(x, y):
    return train_test_split(x, y, test_size=0.2, random_state=42)

def create_unfederated_dataset(x, features):
    former_shape = x[:, 1:features].shape
    client_x = np.delete( x[:, 1:features], 2, 1 ).reshape(former_shape[0], former_shape[1]-1)
    #scale 
    client_x = scaler.transform(client_x)
    client_y = x[:, 3].reshape(-1, 1)
    X_train, X_test, y_train, y_test = get_split(client_x, client_y)
    print(len(X_train))
    train_data = create_dataset(X_train, y_train, SHUFFLE_BUFFER, BATCH_SIZE, use_tff=False)
    test_data = create_dataset(X_test, y_test)
    return train_data, test_data

In [ ]:
config = read_config()
BATCH_SIZE = config["BATCH_SIZE"]
PREFETCH_BUFFER = config["PREFETCH_BUFFER"]
SHUFFLE_BUFFER = config["SHUFFLE_BUFFER"]
CLIENTS = config["CLIENTS"]
DATA_DIR = config["DATA_DIR"]
OUT_DIR = config["OUT_DIR"]
LOG_DIR = config["LOG_DIR"]
EPOCHS = config["EPOCHS"] 
NUM_CLASSES = config["NUM_CLASSES"]
file = config["file_top_apps"]

logging.basicConfig(filename= LOG_DIR + 'TFF.log', level=logging.DEBUG)
if IN_COLAB:
    tf_log_dir = "/tmp/logs/scalars/training/"
    !rm -R /tmp/logs/scalars/*

else:
    tf_log_dir = LOG_DIR + "tensorboard/"
summary_writer = tf.summary.create_file_writer(tf_log_dir)

In [ ]:
entropy_loss = tf.keras.losses.SparseCategoricalCrossentropy()
sparseCategoricalAcc = tf.keras.metrics.SparseCategoricalAccuracy()
sparseTopKCategoricalAccuracy = tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5)
client_optimizer =  tf.keras.optimizers.SGD(learning_rate= 0.6, momentum=0.6, nesterov=True)
server_optimzer = tf.keras.optimizers.SGD(learning_rate= 1.0)

In [ ]:
# from client_manager
train_data = []
test_data = []
client_ids = None
use_tff = True
scaler = StandardScaler()
utils = Utils()
reader = Reader(BASE_DIR + DATA_DIR, file)
data = reader.get_data()

if ("IID" in file): 
    data = utils.create_clients(data, CLIENTSs, strict = False)
    reader.set_features(reader.get_features() + 1)
    client_ids =  [i for i in range(0, CLIENTS)]
cols = [i for i in range(0, reader.get_features())]
del cols[0]
del cols[2]

features = reader.get_features()
scaler.fit(data[:, cols])
if ((file == "App_usage_trace.txt") or (file == "top_90_apps.csv")): 
    data  = utils.map_ids(data.copy())
    num_of_users = int((np.amax(data[:, 0]) + 1))
    client_ids = list(range(0, num_of_users))
    logging.info(len(client_ids))
    random.shuffle(client_ids)
    client_ids = client_ids[:CLIENTS]
client_ids = sorted(client_ids)
for id in client_ids:
    indicees = data[:, 0] == id
    former_shape = data[indicees, 1:features].shape
    #delete index 0 and 3, containing the label and the user id
    client_x = np.delete( data[indicees, 1:features], 2, 1 ).reshape(former_shape[0], former_shape[1]-1)
    #scale 
    client_x = scaler.transform(client_x)
    client_y = data[indicees, 3].reshape(-1, 1)
    if len(client_x) > 1:
        X_train, X_test, y_train, y_test = train_test_split(client_x, client_y, test_size=0.2, random_state=42)    
        ds_train = create_dataset(X_train, y_train, use_tff)
        ds_test = create_dataset(X_test, y_test, use_tff)
        print("Client {}: Created  dataset".format(id))

        train_data.append(ds_train)
        test_data.append(ds_test)
    else:
        print("Could not generate datasets for client {} as there is just one entry in X_train".format(id))
        client_ids.remove(id)

In [ ]:
# Check format for TFF: needs to be in shape(None, dim)
#like eg:
# (TensorSpec(shape=(None, 3), dtype=tf.float64, name=None),
#  TensorSpec(shape=(None, 1), dtype=tf.float64, name=None)
print(train_data[0].element_spec)
print(test_data[0].element_spec)

In [ ]:
def create_keras_model(input_dim=3):
    return tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(input_dim,)),
      tf.keras.layers.Dense(500, activation=tf.nn.relu),
      tf.keras.layers.Dense(NUM_CLASSES, activation='softmax'),
    ])

# Each time the next method is called, the server model is broadcast to each client using a broadcast function. 
# For each client, one epoch of local training is performed via the tf.keras.optimizers.Optimizer.apply_gradients method of the client optimizer. 
# Each client computes the difference between the client model after training and the initial broadcast model. 
# These model deltas are then aggregated at the server using some aggregation function. 
# The aggregate model delta is applied at the server by using the tf.keras.optimizers.Optimizer.apply_gradients method of the server optimizer.
def model_fn():
  # We _must_ create a new model here, and _not_ capture it from an external
  # scope. TFF will call this within different graph contexts.
    keras_model = create_keras_model(3)
    return tff.learning.from_keras_model(
      keras_model,
      input_spec = train_data[0].element_spec,
      loss = tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics = [tf.keras.metrics.SparseCategoricalAccuracy(), 
               tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5)]) 
  
def run_federated():
    with tf.device('/gpu:0'):
    iterative_process = tff.learning.build_federated_averaging_process(
        model_fn,
        client_optimizer_fn = lambda: tf.keras.optimizers.SGD(learning_rate= 0.6, momentum=0.6, nesterov=True), 
        server_optimizer_fn = lambda: tf.keras.optimizers.SGD(learning_rate= 1.0)
        )

    state = iterative_process.initialize()
    with summary_writer.as_default():
        for round_num in range(EPOCHS):
            state, metrics = iterative_process.next(state, train_data)

            # Note: training metrics reported by the iterative training process 
            #generally reflect the performance of the model at the beginning of the training round
            for name, value in metrics['train'].items():
                tf.summary.scalar(name, value, step=round_num)
                print(round_num, name, value)

    evaluation = tff.learning.build_federated_evaluation(model_fn)  
    test_metrics = evaluation(state.model, test_data)
    print(f"{round_num} Test: {test_metrics}")

In [ ]:
run_federated()